In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree
import pickle

In [2]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [3]:
housing= pd.read_csv('train.csv')
housing_test= pd.read_csv('test.csv')

In [4]:
sale_price = housing['SalePrice']
min_sales = housing['SalePrice'].min()
max_sales = housing['SalePrice'].max()
print(min_sales, max_sales)

34900 755000


In [5]:
#norm_df = (df - _min) / (_max - _min) # 0-1
#scale = df * (_max - _min) + _min # 
norm_sales = (housing['SalePrice'] - min_sales) / (max_sales - min_sales)
norm_sales

0       0.241078
1       0.203583
2       0.261908
3       0.145952
4       0.298709
          ...   
1455    0.194556
1456    0.243161
1457    0.321622
1458    0.148903
1459    0.156367
Name: SalePrice, Length: 1460, dtype: float64

In [6]:
#housing['YrSold'].value_counts()

In [7]:
house_cleaning_1 = housing.replace(['Grvl', 'Pave', 'Gd', 'Fa', 'EX', 'MnPrv', 'GdPrv', 'GdWo', 'GdWw', 'Shed', 'Othr', 'Gar2', 'Tenc'], [1, 1, 1, 1, 1,1 ,1 ,1, 1,1 ,1 , 1, 1])
house_cleaning_2 = house_cleaning_1.fillna(value = 0)
house_cleaning = house_cleaning_2.drop(columns = ['SalePrice'])

In [8]:
X = house_cleaning
y = norm_sales.values.reshape(-1, 1)
print(X.shape, y.shape)

(1460, 80) (1460, 1)


In [9]:
test= pd.get_dummies(X)
drops_test = test[['OverallCond', 'FullBath', 'BedroomAbvGr', '1stFlrSF','YearBuilt',
                               'HalfBath','2ndFlrSF', 'GarageArea', 'LotArea', 'GarageCars', 'KitchenAbvGr' ]]
#drops_test.to_csv(r"testing.csv", index=False)


In [10]:
norm_sales.head()

0    0.241078
1    0.203583
2    0.261908
3    0.145952
4    0.298709
Name: SalePrice, dtype: float64

In [11]:
#iris = load_iris()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(drops_test, y , random_state=42)


In [13]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7163666480537733

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

C:\Users\NCDSB\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.8692933503290282

In [15]:
sorted(zip(rf.feature_importances_, drops_test), reverse=True)

[(0.31422485144808676, 'GarageCars'),
 (0.18500427599385602, 'YearBuilt'),
 (0.1698305798715469, '1stFlrSF'),
 (0.10171150957339119, '2ndFlrSF'),
 (0.0653804935181497, 'LotArea'),
 (0.05826212224928963, 'FullBath'),
 (0.045391304258527984, 'GarageArea'),
 (0.018249168950418514, 'OverallCond'),
 (0.016757947336770145, 'BedroomAbvGr'),
 (0.012916466989515209, 'KitchenAbvGr'),
 (0.012271279810447958, 'HalfBath')]

In [16]:
rf.fit(X_train, y_train)

C:\Users\NCDSB\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(n_estimators=200)

In [17]:
predictions = rf.predict(drops_test)
#print(predictions)
#print(drops_test)

In [18]:
#first_row = rf.predict([[5, 2, 3, 856, 2003, 1, 854, 548, 8450, 2, 1]])
#print(first_row)

In [19]:
reverse = ((max_sales - min_sales)*(predictions))+ min_sales
print(reverse)

[207609.705 176707.5   221905.655 ... 242022.    133848.5   147256.125]


In [20]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

print(mse)
print(r2)

0.0006970719552955074
0.9426867601369813


In [21]:
pkl_filename = "housing.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(rf, file)

In [22]:
#with open(pkl_filename, 'rb') as file:
    #pickle_model = pickle.load(file)